# Config

## Libraries

In [26]:
import pandas as pd
import requests
import json
import plotly.graph_objects as go
from IPython.core.display import HTML

## Environment

In [2]:
domains = {
    "dev": "https://dev-selfdecode.com",
    "qa": "https://qa-selfdecode.com",
    "qa-b2b": "https://qa-b2b.selfdecode.com",
    "prod": "https://selfdecode.com",
    "prod-eu": "https://eu.selfdecode.com"
}

In [3]:
tokens = pd.read_csv('tokens2.csv', index_col=['company', 'env'])

In [85]:
env = 'qa'
company = 'arquimea'
language = 'es'

DOMAIN = domains[env]
auth_token = "Token " + tokens.loc[company, env].item()
cookies = {"sd-lang": language}

headers = {"Accept": "application/json",
           "Content-Type": "application/json",
           "Authorization": auth_token}

# Profile

In [12]:
def get_profiles(profile_id=''):
    url = f"{DOMAIN}/service/profiles/profile/{profile_id}"
    
    response = requests.get(url,
                            headers = headers)
    response.raise_for_status()
    return response.json()

In [86]:
profiles_df = pd.json_normalize(get_profiles())
profiles_df

,id,user_id,name,ethnicity,secondary_ethnicity,sex,birth_day,birth_month,birth_year,has_data_missing,dna_sample_id,is_v2,created_at,email,height,weight,created_by_migration
0,cfb252fc-503b-458d-a2b6-14f73e27df20,cc81c558-0484-47cf-85a4-53a97ce1da0b,David Fernández (2022-11-03T12:55:54.680448),White or European,None,Male,17.0,9.0,1976,False,None,True,1.667480e+09,davidf.76+epi@gmail.com,180.0,86.0,False
1,58aabe53-47b6-4d20-9088-e0375f3d6d60,cc81c558-0484-47cf-85a4-53a97ce1da0b,Test Arq,White or European,None,Male,NaN,NaN,1976,False,None,True,1.669631e+09,davidf.76+testA@gmail.com,180.0,85.5,False
2,8aa45745-0442-43cb-9057-16c1977785d4,cc81c558-0484-47cf-85a4-53a97ce1da0b,Carlos M M (2023-03-08T16:43:54.221674500),White or European,None,Female,12.0,2.0,1994,False,None,True,1.678294e+09,cmartin@arquimea.com,180.0,85.5,False
3,b27f62dc-7c67-4b12-985d-20be15f982f6,cc81c558-0484-47cf-85a4-53a97ce1da0b,Carlos Martin Mateos (2023-03-13T12:19:42.7174...,White or European,None,Female,22.0,3.0,1994,False,None,True,1.678710e+09,cmartin+1@arquimea.com,175.0,70.0,False
4,32d882ae-ea07-4bd0-96c2-4acd531129cd,cc81c558-0484-47cf-85a4-53a97ce1da0b,Carlos MArtin Mate (2023-03-16T11:54:46.148680...,White or European,None,Female,12.0,2.0,1994,False,None,True,1.678968e+09,cmartin+2@arquimea.com,180.0,80.0,False


In [87]:
profile_id = profiles_df.iloc[0]['id']
profile_id

'cfb252fc-503b-458d-a2b6-14f73e27df20'

# Reports

## Report structure

In [5]:
report_structure = {"sections": [{"name": "Salud Mental",
                                  "reports": [{"id": "07a6a195-826e-4118-8179-2d9e72ff0069",
                                               "name": "Memoria"},
                                              {"id": "e525cc4d-5e3d-4858-bac8-276d7e3e72ce",
                                               "name": "Deterioro Cognitivo"},
                                              {"id": "305b4f9d-6895-48e2-be8b-161819ce5b9b",
                                               "name": "Stress"},
                                              {"id": "588dfca3-5087-4c7c-9ef5-543393789062",
                                               "name": "Adicciones"},
                                              {"id": "70f4835b-2eab-462e-9261-20bba8159419",
                                               "name": "Atención"},
                                              {"id": "4523567c-4440-43e2-b029-ea3dbf3f908d",
                                               "name": "Velocidad de procesamiento"}]},
                                 {"name": "Cardiovascular",
                                  "reports": [{"id": "ff4f5b07-89fe-4a02-a151-60ff0de8b721",
                                               "name": "Sensibilidad a la Sal"},
                                              {"id": "eafcc215-e36d-4493-bfd5-62bae419ee4b",
                                               "name": "Endurecimiento Arterial"},
                                              {"id": "6d7eb27c-ccc7-4ee0-af55-30913ec48462",
                                               "name": "Palpitaciones"}]},
                                 {"name": "Bienestar y Descanso",
                                  "reports": [{"id": "6ba15dec-a7ef-448f-9bd9-21c054f49c0e",
                                               "name": "Fatiga"},
                                              {"id": "3c443790-ac9c-471f-a60c-a04f6464320f",
                                               "name": "Dolor cronico"},
                                              {"id": "39bd8e00-5c1b-4280-9618-81baa829a740",
                                               "name": "Insomnio"},
                                              {"id": "557a683d-bd9a-4840-a652-084651677a28",
                                               "name": "Apnea del Sueño"},
                                              {"id": "b9dc7199-1f56-4344-b9b3-8c978bcff206",
                                               "name": "Somnolencia diurna"}]},
                                 {"name": "Sistema Urinario",
                                  "reports": [
                                      {"id": "12479cf0-736c-47b5-abe8-40c765cb1164",
                                       "name": "Vejiga hiperactiva"},
                                      {"id": "ccfec08f-3a76-40c6-ae88-407761311b0d",
                                       "name": "Infecciones Urinarias de repetición"}]}]}

## Generate missing reports for selected profile

In [11]:
def get_profile_reports(profile_id, report_id=None):
    url = f"{DOMAIN}/service/reports/profile-report/?profile_id={profile_id}"
    
    if report_id:
        url = url + '&report_id=' + report_id
        
    response = requests.get(url,
                            headers=headers)
    response.raise_for_status()
    return response.json()

In [80]:
profile_reports = get_profile_reports(profile_id)
profile_reports

[]

In [82]:
reports_to_generate = []
for section in report_structure['sections']:
    for report in section['reports']:
        if not any(profile_report.get('report_id') == report['id']
                   for profile_report in profile_reports):
            reports_to_generate.append(report['id'])

reports_to_generate

['07a6a195-826e-4118-8179-2d9e72ff0069',
 'e525cc4d-5e3d-4858-bac8-276d7e3e72ce',
 '305b4f9d-6895-48e2-be8b-161819ce5b9b',
 '588dfca3-5087-4c7c-9ef5-543393789062',
 '70f4835b-2eab-462e-9261-20bba8159419',
 '4523567c-4440-43e2-b029-ea3dbf3f908d',
 'ff4f5b07-89fe-4a02-a151-60ff0de8b721',
 'eafcc215-e36d-4493-bfd5-62bae419ee4b',
 '6d7eb27c-ccc7-4ee0-af55-30913ec48462',
 '6ba15dec-a7ef-448f-9bd9-21c054f49c0e',
 '3c443790-ac9c-471f-a60c-a04f6464320f',
 '39bd8e00-5c1b-4280-9618-81baa829a740',
 '557a683d-bd9a-4840-a652-084651677a28',
 'b9dc7199-1f56-4344-b9b3-8c978bcff206',
 '12479cf0-736c-47b5-abe8-40c765cb1164',
 'ccfec08f-3a76-40c6-ae88-407761311b0d']

In [18]:
def generate_profile_reports(profile_id, report_ids):
    url = f"{DOMAIN}/service/reports/profile-report/"
    
    payload = []
    for report_id in report_ids:
        payload_dict = {'profile_id': profile_id, 'report_id': report_id}
        payload.append(payload_dict)
    response = requests.post(url,
                            json=payload,
                            headers=headers)
    response.raise_for_status()
    return response.json()

In [83]:
generate_profile_reports(profile_id, reports_to_generate)

[{'id': '4ac0505f-b85d-4d8f-a1fe-5ece778530d9',
  'profile_id': '4d8b2526-fe5f-4403-a884-8c8f121727db',
  'report_id': '07a6a195-826e-4118-8179-2d9e72ff0069',
  'status': 'processing',
  'fail_notice_viewed': False,
  'version': '1.0.0',
  'purchased': False,
  'is_readable': True,
  'has_read': True,
  'was_generated': False,
  'is_new': False,
  'last_generated_at': None},
 {'id': 'c9d20b16-d5c6-4fd4-8887-9ef2b65314b6',
  'profile_id': '4d8b2526-fe5f-4403-a884-8c8f121727db',
  'report_id': 'e525cc4d-5e3d-4858-bac8-276d7e3e72ce',
  'status': 'processing',
  'fail_notice_viewed': False,
  'version': '1.0.0',
  'purchased': False,
  'is_readable': True,
  'has_read': True,
  'was_generated': False,
  'is_new': False,
  'last_generated_at': None},
 {'id': '43b484a8-42e0-4add-8563-5b1daa3838a6',
  'profile_id': '4d8b2526-fe5f-4403-a884-8c8f121727db',
  'report_id': '305b4f9d-6895-48e2-be8b-161819ce5b9b',
  'status': 'processing',
  'fail_notice_viewed': False,
  'version': '1.0.0',
  'pur

# Render bundled report

In [39]:
def get_profile_trait(profile_report_id):
    url = f"{DOMAIN}/service/reports/profile-trait/?profile_report_id={profile_report_id}"
    response = requests.get(url,
                            headers=headers,
                            cookies=cookies)
    response.raise_for_status()
    return response.json()

In [40]:
def get_profile_recommendation(profile_report_id):
    url = f"{DOMAIN}/service/reports/profile-recommendation/?profile_report_id={profile_report_id}"
    response = requests.get(url,
                            headers=headers,
                            cookies=cookies)
    response.raise_for_status()
    return response.json()

In [41]:
def get_profile_recommendation_trait(profile_report_id):
    url = f"{DOMAIN}/service/reports/profile-recommendation-trait/?profile_report_id={profile_report_id}"
    response = requests.get(url,
                            headers=headers,
                            cookies=cookies)
    response.raise_for_status()
    return response.json()

In [99]:
profile_reports = get_profile_reports(profile_id)

for section in report_structure['sections']:
    display(HTML(f"<h2>{section['name']}</h2>"))
    for report in section['reports']:
        for profile_report in profile_reports:
            if profile_report['report_id'] == report['id']:
                report_status = profile_report['status']
                break
        
        if report_status=='processing':
            display(HTML(f"<h3>{report['name']}</h3>\n"
                         f"Processing"))
        elif report_status=='completed':
            profile_trait = get_profile_trait(profile_report['id'])
            profile_recommendation = get_profile_recommendation(profile_report['id'])
            
            display(HTML(f"<h3>{profile_trait[0]['trait']['name']}</h3>\n"
                         f"Your score: {profile_trait[0]['score']}<br>\n"
                         f"{profile_trait[0]['trait_score_text']}<br>\n"
                         f"<h4>Summary</h4>\n"
                         f"{profile_trait[0]['trait']['summary']}<br>\n"
                         f"<h4>{profile_trait[0]['trait']['text_header']}</h4>\n"
                         f"{profile_trait[0]['trait']['text']}<br>\n"
                         f"<br>\n"
                         f"<h4>Recommendations</h4>"))
            
            for rec in profile_recommendation:
                display(HTML(f"<h5>{rec['recommendation']['name']}</h5>\n"
                             f"{rec['recommendation']['sentence']}<br>\n"
                             f"<table>\n"
                             f"<tr>\n"
                             f"<th>Type</th>\n"
                             f"<th>Effect</th>\n"
                             f"<th>Evidence</th>\n"
                             f"</tr>\n"
                             f"<tr>\n"
                             f"<td>{rec['recommendation']['recommendation_type']}</td>\n"
                             f"<td>{rec['effect']}</td>\n"
                             f"<td>{rec['evidence']}</td>\n"
                             f"</tr>\n"
                             f"</table>\n"
                             f"{rec['recommendation']['generic_text']}<br>\n"))

Type,Effect,Evidence
lifestyle,0.8,1.0


Type,Effect,Evidence
lifestyle,0.6,0.8


Type,Effect,Evidence
lifestyle,0.6,0.8


Type,Effect,Evidence
lifestyle,0.4,0.6


Type,Effect,Evidence
lifestyle,0.4,0.6


...

Type,Effect,Evidence
supplement,0.0,0.0


Type,Effect,Evidence
supplement,0.0,0.0


Type,Effect,Evidence
supplement,0.0,0.0


Type,Effect,Evidence
supplement,0.0,0.0


Type,Effect,Evidence
supplement,0.0,0.0
